SyntaxError: invalid non-printable character U+00A0 (1786199312.py, line 12)

In [3]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from dotenv import load_dotenv
import os

load_dotenv("home/srv/Lab/RW/.env")  # Загружаем переменные из .env

DATABASE_URL = os.getenv("DATABASE_URL")
engine = create_engine(DATABASE_URL)
SessionLocal = sessionmaker(autocommit=False, autoflush=False, bind=engine)

# Проверка подключения
with engine.connect() as conn:
    print("Успешное подключение к PostgreSQL!")

ModuleNotFoundError: No module named 'sqlalchemy'

In [3]:
from app.database import engine
print(engine.url)  # Должно показать postgresql://.../base_1

ModuleNotFoundError: No module named 'app.database'

In [6]:
from app.database import SessionLocal, engine
from app.models import Client, Genre, Object  # Импортируем все модели

# Создаем тестовые данные
def create_test_data():
    db = SessionLocal()
    
    try:
        # Тестовый жанр
        genre = Genre(name="Fantasy", description="Magical worlds")
        db.add(genre)
        
        # Тестовый объект
        obj = Object(
            name="The Lord of the Rings",
            year=1954,
            description="Epic fantasy novel",
            genre=genre
        )
        db.add(obj)
        
        db.commit()
        print("Тестовые данные созданы!")
    except Exception as e:
        db.rollback()
        print(f"Ошибка: {e}")
    finally:
        db.close()

create_test_data()

Ошибка: (psycopg2.errors.UniqueViolation) duplicate key value violates unique constraint "genres_name_key"
DETAIL:  Key (name)=(Fantasy) already exists.

[SQL: INSERT INTO genres (name, description) VALUES (%(name)s, %(description)s) RETURNING genres.id]
[parameters: {'name': 'Fantasy', 'description': 'Magical worlds'}]
(Background on this error at: https://sqlalche.me/e/20/gkpj)


In [7]:
from app.database import SessionLocal
from app.models import Genre, Object

def show_test_data():
    db = SessionLocal()
    try:
        # Получаем все жанры
        genres = db.query(Genre).all()
        print("Жанры:")
        for genre in genres:
            print(f"- {genre.name}: {genre.description}")
        
        # Получаем все объекты
        objects = db.query(Object).all()
        print("\nОбъекты:")
        for obj in objects:
            print(f"- {obj.name} ({obj.year}): {obj.description}")
            print(f"  Жанр: {obj.genre.name if obj.genre else 'Нет жанра'}")
            
    finally:
        db.close()

show_test_data()

Жанры:
- Fantasy: Magical worlds

Объекты:
- The Lord of the Rings (1954): Epic fantasy novel
  Жанр: Fantasy


In [8]:
from sqlalchemy import select

db = SessionLocal()
fantasy_books = db.scalars(
    select(Object).join(Genre).where(Genre.name == "Fantasy")
).all()

for book in fantasy_books:
    print(f"{book.name} ({book.year})")

The Lord of the Rings (1954)


In [9]:
first_obj = db.get(Object, 1)  # Получить объект с id=1
if first_obj:
    print(f"Жанр объекта '{first_obj.name}': {first_obj.genre.name}")

Жанр объекта 'The Lord of the Rings': Fantasy


In [12]:
from app.models import Client
from app.database import SessionLocal

# Создаем сессию
db = SessionLocal()

# Тестируем методы
test_client = Client(login="test_user")
test_client.set_password("my_secure_password")  # Теперь метод доступен

# Проверяем хеш
print(f"Хеш пароля: {test_client.password_hash}")

# Проверяем проверку пароля
print("Проверка правильного пароля:", test_client.check_password("my_secure_password"))
print("Проверка неправильного пароля:", test_client.check_password("wrong_password"))

db.close()

NameError: name 'os' is not defined

In [11]:
from app.database import SessionLocal
from app.models import Client

def create_test_user():
    db = SessionLocal()
    try:
        # Создаем объект пользователя
        test_user = Client(
            login="test_user",
        )
        
        # Устанавливаем хеш пароля
        test_user.set_password("passTester1337")  # Используем наш метод
        
        # Добавляем и сохраняем
        db.add(test_user)
        db.commit()
        print(f"Пользователь {test_user.login} создан! ID: {test_user.id}")
        
        # Проверяем пароль
        print("Проверка пароля:", test_user.check_password("test_password123"))
        
        return test_user
    except Exception as e:
        db.rollback()
        print(f"Ошибка: {e}")
        raise
    finally:
        db.close()

# Вызываем функцию
test_user = create_test_user()

Ошибка: name 'os' is not defined


NameError: name 'os' is not defined

In [9]:
from app import create_app

app = create_app()

if __name__ == "__main__":
    app.run(debug=True)

ImportError: cannot import name 'create_app' from 'app' (/home/srv/Lab/RW/app/__init__.py)